In [1]:
import pandas as pd
import os

In [2]:
local_modules_path = '../src'
os.sys.path.append(local_modules_path)

In [3]:
import api_fetcher

This notebooks was created with the aim of visualizing Tennis demand and create a simple ML model

First, how can we get this data? We are able to pull data from the Amazon's public API via rapidapi. So, we need to search in the product categories, where the Tennis is. It might be likely needed to pull both Women and Men Tenis. Let's take a look at the Women Tenis

In [4]:
amazon_api_client = api_fetcher.ApiClient()
amazon_api_client.http_request(
    method="GET",
    request_url="/products-by-category?category_id=2478868012&page=1&country=US&sort_by=RELEVANCE&product_condition=ALL&is_prime=false&deals_and_discounts=NONE"
)
amazon_api_client.save_response(filename='product_list_us')

In [5]:
import json

with open('../data/api-calls/us_product_list.json', 'r') as file:
    product_list_json = json.load(file)

In [6]:
api_call_response = eval(product_list_json['response'])

In [7]:
api_call_data = api_call_response['data']

In [8]:
pd.DataFrame(api_call_data).sort_values(by="name")

,name,id
12,AWS Courses,courses
1,Alexa Skills,alexa-skills
0,All Departments,aps
2,Amazon Devices,amazon-devices
3,Amazon Explore,live-explorations
4,Amazon Fresh,amazonfresh
5,Amazon Pharmacy,amazon-pharmacy
6,Amazon Warehouse,warehouse-deals
7,Appliances,appliances
8,Apps & Games,mobile-apps


It seems we need to delve depet into to both `fashion-mens` and `fashion-womens`

Let's try out with the path parameter `product-details`

In [72]:
amazon_api_client.http_request(
    method="GET",
    request_url="/search?query=Tenis&country=US&sort_by=RELEVANCE&product_condition=ALL&page=1"
)

{'method': 'GET',
 'request_url': '/search?query=Tenis&country=US&sort_by=RELEVANCE&product_condition=ALL&page=1',
 'response': '{"status":"OK","request_id":"a4a9a546-1f41-4d04-8a31-4eded5c1dfb0","parameters":{"query":"Tenis","country":"US","sort_by":"RELEVANCE","page":1},"data":{"total_products":43203,"country":"US","domain":"www.amazon.com","products":[{"asin":"B0BZXSLD5H","product_title":"Under Armour men&#x27;s Charged Commit Trainer 4 Sneaker","product_price":"$39.20","product_original_price":"$80.00","currency":"USD","product_star_rating":"4.6","product_num_ratings":394,"product_url":"https://www.amazon.com/dp/B0BZXSLD5H","product_photo":"https://m.media-amazon.com/images/I/51Ih-2zQ++L._AC_UL960_FMwebp_QL65_.jpg","product_num_offers":1,"product_minimum_offer_price":"$39.20","is_best_seller":false,"is_amazon_choice":false,"is_prime":true,"climate_pledge_friendly":false,"sales_volume":"50+ bought in past month","delivery":"FREE delivery Fri, Oct 11","has_variations":true},{"asin":"

In [73]:
amazon_api_client.save_response(filename="tenis_products_all")

In [75]:
with open(
    "../data/api-calls/tenis_products_all.json",
    'r'
) as file:
    prod_details_json = json.load(file)

In [76]:
product_data = json.loads(prod_details_json['response'])['data']

In [93]:
h_request = amazon_api_client.http_request(method="GET", request_url="/search?asin=B077HQTBT6&country=US")

In [94]:
h_request

{'method': 'GET',
 'request_url': '/search?asin=B077HQTBT6&country=US',
 'response': '{"status":"OK","request_id":"0ca6cd95-8413-4331-8113-a0c464b98998","parameters":{"query":"","country":"US","sort_by":"RELEVANCE","page":1},"data":{"total_products":0,"country":"US","domain":"www.amazon.com","products":[]}}'}

In [87]:
tenis_data = pd.DataFrame(product_data['products'])

In [91]:
tenis_data.iloc[0]['sales_volume']

'50+ bought in past month'

In [90]:
tenis_data['sales_volume'].unique()

array(['50+ bought in past month', '100+ bought in past month', 'List: ',
       '200+ bought in past month', '300+ bought in past month', None,
       '900+ bought in past month', '800+ bought in past month',
       '500+ bought in past month', 'No featured offers available',
       '1K+ bought in past month'], dtype=object)

In [71]:
product_data['total_products']

KeyError: 'total_products'

In [80]:
product_data

{'total_products': 43203,
 'country': 'US',
 'domain': 'www.amazon.com',
 'products': [{'asin': 'B0BZXSLD5H',
   'product_title': 'Under Armour men&#x27;s Charged Commit Trainer 4 Sneaker',
   'product_price': '$39.20',
   'product_original_price': '$80.00',
   'currency': 'USD',
   'product_star_rating': '4.6',
   'product_num_ratings': 394,
   'product_url': 'https://www.amazon.com/dp/B0BZXSLD5H',
   'product_photo': 'https://m.media-amazon.com/images/I/51Ih-2zQ++L._AC_UL960_FMwebp_QL65_.jpg',
   'product_num_offers': 1,
   'product_minimum_offer_price': '$39.20',
   'is_best_seller': False,
   'is_amazon_choice': False,
   'is_prime': True,
   'climate_pledge_friendly': False,
   'sales_volume': '50+ bought in past month',
   'delivery': 'FREE delivery Fri, Oct 11',
   'has_variations': True},
  {'asin': 'B09ZF98TZQ',
   'product_title': 'Reebok Unisex Glide Sneaker',
   'product_price': '$39.00',
   'product_original_price': '$70.00',
   'currency': 'USD',
   'product_star_rating':

In [44]:
raw_tenis_data = str(product_data)

In [45]:
import re

In [57]:
regexp_search = re.search(pattern="Under", string=raw_tenis_data)

In [58]:
regexp_search

<re.Match object; span=(41, 46), match='Under'>

In [35]:
prod_details_df = pd.DataFrame(
    data=product_data
)

ValueError: All arrays must be of the same length

In [28]:
prod_details_df

,name,id
0,All Departments,aps
1,Alexa Skills,alexa-skills
2,Amazon Devices,amazon-devices
3,Amazon Explore,live-explorations
4,Amazon Fresh,amazonfresh
5,Amazon Pharmacy,amazon-pharmacy
6,Amazon Warehouse,warehouse-deals
7,Appliances,appliances
8,Apps & Games,mobile-apps
9,"Arts, Crafts & Sewing",arts-crafts


In [26]:
import http.client

conn = http.client.HTTPSConnection("real-time-amazon-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "dfa8842b83msha2bc48dbc5792bdp1cbbd0jsn0bde68569041",
    'x-rapidapi-host': "real-time-amazon-data.p.rapidapi.com"
}

conn.request("GET", "/product-details?asin=B07ZPKBL9V&country=US", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"OK","request_id":"550bcf63-d4bb-47cb-a00d-8f9ab242c13c","parameters":{"asin":"B07ZPKBL9V","country":"US"},"data":{"asin":"B07ZPKBL9V","product_title":"Apple iPhone 11, 64GB, (PRODUCT)RED - Fully Unlocked (Renewed)","product_price":"$222.00","product_original_price":null,"currency":"USD","country":"US","product_byline":"Visit the Amazon Renewed Store","product_star_rating":"4.2","product_num_ratings":53023,"product_url":"https://www.amazon.com/dp/B07ZPKBL9V","product_photo":"https://m.media-amazon.com/images/I/514k7uOBMwL._AC_SL1000_.jpg","product_num_offers":32,"product_availability":"In Stock","is_best_seller":false,"is_amazon_choice":false,"is_prime":false,"climate_pledge_friendly":true,"sales_volume":"500+ bought in past month","about_product":["This phone is unlocked and compatible with any carrier of choice on GSM and CDMA networks (e.g. AT&T, T-Mobile, Sprint, Verizon, US Cellular, Cricket, Metro, Tracfone, Mint Mobile, etc.).","Tested for battery health and guaranteed